# Preparación de datos de test

In [ ]:
import pickle as pckl

In [ ]:
input_path = "/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-new-parsed/data_train_night.txt"
output_path = "/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-new-parsed/data_test_night.pckl"
mapper = {0:'Panel', 1:'Dedo'}

In [ ]:
rows = []
with open(input_path) as fd:
    for item in fd:
        filename_and_boxes = item.rstrip('\n').split(' ')
        filename = filename_and_boxes[0]
        boxes = filename_and_boxes[1:]
        d = {'filename': filename, 'object':[]}
        for box in boxes:
            box = box.split(',')
            d['object'].append({'xmin':int(box[0]), 'ymin':int(box[1]), 'xmax': int(box[2]), 'ymax': int(box[3]), 'name': mapper[int(box[4])]})
        rows.append(d)

In [ ]:
pckl.dump(rows, open(output_path, 'wb'))

# Dependencias

In [1]:
import argparse
import json
import pickle as pckl
import numpy as np
import os
import cv2
import pandas as pd
from PIL import Image
from scipy.special import expit
from yolo3.yolo import YOLO
from tqdm import tqdm

## Cargar modelo

In [2]:
def load_model(model_path, classes_path, anchors_path):

    yolo = YOLO(
        **{
            "model_path": model_path,
            "anchors_path": anchors_path,
            "classes_path": classes_path,
            "score": 0.5,
            "gpu_num": 1,
            "model_image_size": (416, 416),
        }
    )

    return yolo

## Bounding boxes

In [3]:
class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, c = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        
        self.c       = c
        self.classes = classes

        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)
        
        return self.label
    
    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
            
        return self.score 

def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
             return 0
        else:
            return min(x2,x4) - x3  
        
def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])  
    
    intersect = intersect_w * intersect_h

    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    
    union = w1*h1 + w2*h2 - intersect
    
    return float(intersect) / union

## Generador de lotes

In [4]:
class BatchGenerator():
    def __init__(self, instances, anchors, labels, batch_size=1, shuffle=True):
        self.instances          = instances
        self.batch_size         = batch_size
        self.labels             = labels
        self.anchors            = [BoundBox(0, 0, anchors[2*i], anchors[2*i+1]) for i in range(len(anchors)//2)]

        if shuffle:
            np.random.shuffle(self.instances)      
            
    def num_classes(self):
        return len(self.labels)

    def size(self):
        return len(self.instances)    

    def get_anchors(self):
        anchors = []

        for anchor in self.anchors:
            anchors += [anchor.xmax, anchor.ymax]

        return anchors

    def load_annotation(self, i):
        annots = []

        for obj in self.instances[i]['object']:
            annot = [obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax'], self.labels.index(obj['name'])]
            annots += [annot]

        if len(annots) == 0: annots = [[]]

        return np.array(annots)

    def load_image(self, i):
        return cv2.imread(self.instances[i]['filename'])    

## Detection

In [5]:
def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
    else:
        return
        
    for c in range(nb_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])

        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]

            if boxes[index_i].classes[c] == 0: continue

            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]

                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0
                    
def get_yolo_boxes(model, images, net_h, net_w, nms_thresh):
    batch_output, data = model.detect_image(Image.fromarray(images[0].astype('uint8')))
    boxes = []
    
    for bo in batch_output:
        b = [0]*2
        b[bo[4]] = bo[5]
        box = bo[:4] + [bo[5]] + [b]
        boxes.append(BoundBox(box[0], box[1], box[2], box[3], box[4], box[5]))

    # image_h, image_w, _ = images[0].shape
    # correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w)

    do_nms(boxes, nms_thresh)
    return [boxes]


In [6]:
def detection(model, generator, nms_thresh=0.5, net_h=416, net_w=416):  
    # gather all detections and annotations
    all_detections     = [[None for i in range(generator.num_classes())] for j in range(generator.size())]
    all_annotations    = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in range(generator.size()):
        raw_image = [generator.load_image(i)]

        # make the boxes and the labels
        pred_boxes = get_yolo_boxes(model, raw_image, net_h, net_w, nms_thresh)[0]

        score = np.array([box.get_score() for box in pred_boxes])
        pred_labels = np.array([box.label for box in pred_boxes])        
        
        if len(pred_boxes) > 0:
            pred_boxes = np.array([[box.xmin, box.ymin, box.xmax, box.ymax, box.get_score()] for box in pred_boxes]) 
        else:
            pred_boxes = np.array([[]])  
        
        # sort the boxes and the labels according to scores
        score_sort = np.argsort(-score)
        pred_labels = pred_labels[score_sort]
        pred_boxes  = pred_boxes[score_sort]
        
        # copy detections to all_detections
        for label in range(generator.num_classes()):
            all_detections[i][label] = pred_boxes[pred_labels == label, :]

        annotations = generator.load_annotation(i)
        
        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            all_annotations[i][label] = annotations[annotations[:, 4] == label, :4].copy()
            
    return all_detections, all_annotations

## Evaluation

In [7]:
def compute_overlap(a, b):
    """
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    Parameters
    ----------
    a: (N, 4) ndarray of float
    b: (K, 4) ndarray of float
    Returns
    -------
    overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])

    iw = np.minimum(np.expand_dims(a[:, 2], axis=1), b[:, 2]) - np.maximum(np.expand_dims(a[:, 0], 1), b[:, 0])
    ih = np.minimum(np.expand_dims(a[:, 3], axis=1), b[:, 3]) - np.maximum(np.expand_dims(a[:, 1], 1), b[:, 1])

    iw = np.maximum(iw, 0)
    ih = np.maximum(ih, 0)

    ua = np.expand_dims((a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), axis=1) + area - iw * ih

    ua = np.maximum(ua, np.finfo(float).eps)

    intersection = iw * ih

    return intersection / ua  

def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.

    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap  

In [8]:
def evaluation(all_detections, all_annotations, generator, iou_threshold=0.5):
    average_precisions = []
    
    for label in range(generator.num_classes()):
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0

        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []

            for d in detections:
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0: # Si no hay anotación de esa detección es un falso positivo
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    continue

                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations) # IOU, tiene el consideración todas las anotaciones
                assigned_annotation = np.argmax(overlaps, axis=1) # Se queda con la anotación que maximiza el IOU
                max_overlap         = overlaps[0, assigned_annotation] # Se queda con el valor del IOU se esta anotación

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations: # Comprueba si esa anotación no ha sido ya asignada a una detección (además de comprobar que el IOU supera un cierto umbral). Las detecciones están ordenadas por score descendente por lo que se quedaría primero la que tiene mayor score (aunque luego pueda tener menor IoU).
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation) # Guarda la anotación para que no pueda volver a ser usada
                else: # IOU por debajo del umbral o anotación ya utilizada
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0) 

        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0
            continue

        # sort by score (Esto lo hace para ser consistente con los vectores de anotación y detección)
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]
        annotations_pending = num_annotations - np.sum(true_positives)

        # compute false positives and true positives (Esto es lo mismo que sumar unos y ceros de cada una de los vectores pero se hace así para computar el AP)
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)

        # compute recall and precision (Y el F1)
        recall    = true_positives / num_annotations # Es lo mismo que dividir entre TP + FN porque la suma de ambas tiene que ser el número de anotaciones (se detecten o no)
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        f1 = 2 * (precision * recall) / (precision + recall)

        # compute average precision
        average_precision  = compute_ap(recall, precision)
        average_precisions.append({'label': generator.labels[label], 'AP': average_precision, 'recall': recall[-1] if len(recall) else -1, 'precision': precision[-1] if len(precision) else -1, 'support': num_annotations, 'TP':true_positives[-1] if len(true_positives) else -1, 'FP': false_positives[-1] if len(false_positives) else -1, 'FN': annotations_pending})

    return average_precisions

# Evaluación

## Carga de modelo y de datos de test

In [ ]:
os.chdir('/home/ubuntu/tfm')
config_path = './utils/config.json'

with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

instances = pckl.load(open(config['model']['dataset_folder'], 'rb'))
labels = config['model']['labels']
labels = sorted(labels)

valid_generator = BatchGenerator(
    instances           = instances,
    anchors             = config['model']['anchors'],   
    labels              = sorted(config['model']['labels']),
)

infer_model = load_model(config['train']['model_folder'], config['train']['classes_path'], config['train']['anchors_path'])

## Test

In [ ]:
all_detections, all_annotations = detection(infer_model, valid_generator)

In [ ]:
average_precisions = evaluation(all_detections, all_annotations, valid_generator)

## Procesar salida

In [ ]:
items = 0
precision = 0
for average_precision in average_precisions:    
    items += 1 
    precision += average_precision['AP']
display(pd.DataFrame(average_precisions))
print('mAP: {:.4f}'.format(precision / items))

## Prueba completa

In [9]:
import mlflow
import os
import shutil
import boto3
from datetime import datetime


S3_CLIENT = boto3.resource('s3')

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
MLFLOW_CLIENT = mlflow.tracking.MlflowClient()

REGISTERED_MODELS = ["Hands"]
MODELS = {}


def downlod_model(bucket_name, remoteDirectory_name):

    bucket = S3_CLIENT.Bucket(bucket_name)

    for obj in bucket.objects.filter(Prefix=remoteDirectory_name):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key)


def update_models(version=-1, remove_old_versions=True):

    update = {}

    for model_name in REGISTERED_MODELS:
        model = None
        update[model_name] = 0
        for mv in MLFLOW_CLIENT.search_model_versions(f"name='{model_name}'"):
            mv_bckp = mv
            mv = dict(mv)
            if version == mv['version'] or (version == -1 and mv['current_stage'] == 'Production'):
                mv['last_updated_timestamp'] = str(datetime.fromtimestamp(int(mv['last_updated_timestamp'] / 1000)))
                bucket = mv['source'].split('//')[1].split('/')[0]
                folder = mv['source'].split('//')[1].split('/')[1]
                if os.path.exists(os.path.join('./models', folder)):
                    print("Load existing model...")
                    model = os.path.join(os.path.join('./models', folder), "artifacts/model/data/model.h5")
                else:
                    print("Downloading model...")
                    downlod_model(bucket, folder)
                    model = os.path.join(os.path.join('./models', folder), "artifacts/model/data/model.h5")
                    if remove_old_versions and os.path.exists('./models'):
                        shutil.rmtree('./models')
                    if not os.path.exists('./models'):
                        os.mkdir('./models')
                    shutil.move(os.path.join(os.getcwd(), folder), './models')
                    update[model_name] = 1
                print("Using model {name} v{version} ({current_stage}) updated at {last_updated_timestamp}".format(**mv))
                #response = {k: v for k, v in mv.items() if v}
                break
        if model:
            MODELS[model_name] = (model, mv_bckp)

    return update


def get_model(model_name):

    return MODELS.get(model_name, None)

In [11]:
os.chdir('/home/ubuntu/tfm/standalone')
config_path = '../utils/config.json'

with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

instances = pckl.load(open(config['model']['dataset_folder'], 'rb'))
labels = config['model']['labels']
labels = sorted(labels)

valid_generator = BatchGenerator(
    instances           = instances,
    anchors             = config['model']['anchors'],   
    labels              = sorted(config['model']['labels']),
)

versions = range(13,22)
for version in tqdm(versions):
    update_models(version)
    model_path, model_meta = get_model('Hands')
    infer_model = load_model(model_path, config['train']['classes_path'], config['train']['anchors_path'])
    all_detections, all_annotations = detection(infer_model, valid_generator)
    for iou in [0.6,0.7,0.8,0.9]:
        average_precisions = evaluation(all_detections, all_annotations, valid_generator, iou_threshold=iou)
        items = 0
        precision = 0
        for average_precision in average_precisions:    
            items += 1 
            precision += average_precision['AP']
        pckl.dump(((version,MLFLOW_CLIENT.get_run(model_meta.run_id)),(all_detections, all_annotations), (pd.DataFrame(average_precisions), 'mAP: {:.4f}'.format(precision / items))), open(f"{version}_{iou}_.pckl", 'wb'))

  0%|          | 0/1 [00:00<?, ?it/s]

Using model Hands v14 (None) updated at 2021-01-02 18:26:17
./models/17cc39633ba145419b8568ca9f4b3f88/artifacts/model/data/model.h5 model, anchors, and classes loaded in 22.92sec.
(416, 416, 3)
Found 2 boxes for img
Panel 0.86 (284, 480) (1080, 756)
Dedo 0.97 (575, 679) (854, 1159)
Time spent: 6.176sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.63 (1779, 107) (1894, 326)
Panel 0.95 (389, 139) (1765, 391)
Dedo 0.80 (961, 306) (1120, 736)
Time spent: 2.179sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.62 (709, 67) (891, 336)
Panel 0.63 (0, 0) (632, 343)
Panel 0.72 (860, 202) (1415, 452)
Panel 0.75 (1536, 328) (1911, 581)
Dedo 0.94 (1102, 262) (1352, 867)
Time spent: 2.175sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.87 (510, 172) (1652, 377)
Dedo 0.89 (1017, 272) (1178, 622)
Time spent: 2.195sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.87 (724, 975) (836, 1278)
Time spent: 2.229sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.96 (0, 540) (1080, 815)
Time spent: 2.230sec
(416, 416, 3

Found 2 boxes for img
Panel 0.58 (49, 967) (485, 1140)
Panel 0.65 (25, 533) (552, 872)
Time spent: 2.172sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.93 (138, 132) (1794, 669)
Dedo 0.90 (957, 456) (1136, 743)
Time spent: 2.161sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.97 (467, 208) (1715, 575)
Dedo 0.91 (1037, 383) (1216, 805)
Time spent: 2.148sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.97 (6, 782) (993, 1018)
Dedo 0.95 (589, 926) (714, 1349)
Time spent: 2.166sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.94 (546, 56) (1340, 312)
Panel 0.95 (14, 151) (372, 378)
Dedo 0.94 (1061, 282) (1334, 752)
Time spent: 2.166sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.79 (516, 340) (1477, 559)
Time spent: 2.153sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.58 (324, 115) (1870, 417)
Dedo 0.62 (1238, 263) (1399, 625)
Time spent: 2.157sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.79 (628, 394) (816, 649)
Time spent: 2.162sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (796, 301) (1

Found 1 boxes for img
Dedo 0.89 (829, 357) (1040, 679)
Time spent: 2.158sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.73 (81, 482) (480, 700)
Panel 0.95 (755, 321) (1657, 635)
Dedo 0.76 (1023, 518) (1218, 957)
Time spent: 2.152sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.78 (493, 27) (1218, 712)
Time spent: 2.174sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.97 (221, 88) (1623, 469)
Dedo 0.90 (810, 348) (1022, 637)
Time spent: 2.157sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.50 (385, 257) (1494, 451)
Panel 0.77 (6, 316) (390, 444)
Dedo 0.97 (994, 519) (1301, 1077)
Time spent: 2.186sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.78 (1160, 392) (1920, 731)
Dedo 0.87 (743, 442) (929, 727)
Time spent: 2.171sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.99 (52, 700) (1080, 1036)
Time spent: 2.163sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.89 (107, 800) (1066, 1030)
Dedo 0.89 (681, 970) (864, 1277)
Time spent: 2.172sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.95 (98, 546) (1

Found 3 boxes for img
Panel 0.60 (0, 304) (426, 531)
Panel 0.98 (547, 261) (1451, 497)
Dedo 0.86 (836, 439) (1024, 742)
Time spent: 2.288sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.98 (63, 746) (1079, 1061)
Time spent: 2.179sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.93 (135, 0) (1797, 500)
Dedo 0.96 (608, 349) (779, 649)
Time spent: 2.156sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.89 (873, 209) (1021, 488)
Dedo 0.97 (733, 430) (1012, 910)
Time spent: 2.160sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.66 (441, 232) (1529, 538)
Dedo 0.96 (934, 438) (1100, 728)
Time spent: 2.151sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.82 (271, 301) (507, 558)
Panel 0.85 (785, 394) (1412, 690)
Panel 0.90 (519, 328) (832, 577)
Panel 0.99 (751, 75) (1823, 334)
Dedo 0.91 (675, 490) (817, 787)
Time spent: 2.183sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.99 (526, 14) (1397, 283)
Dedo 0.56 (880, 186) (1022, 496)
Time spent: 2.190sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.95 (176, 1

Found 1 boxes for img
Dedo 0.83 (495, 410) (714, 704)
Time spent: 2.143sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.86 (506, 486) (695, 772)
Time spent: 2.266sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.63 (1021, 229) (1289, 429)
Dedo 0.94 (795, 373) (949, 634)
Time spent: 2.176sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.89 (8, 424) (380, 611)
Panel 0.99 (614, 276) (1552, 597)
Dedo 0.95 (910, 480) (1188, 961)
Time spent: 2.173sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.75 (188, 214) (568, 390)
Dedo 0.90 (272, 312) (431, 655)
Time spent: 2.172sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.96 (443, 155) (1794, 448)
Dedo 0.55 (1072, 324) (1224, 722)
Time spent: 2.166sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.91 (471, 47) (1271, 320)
Panel 0.96 (0, 160) (347, 354)
Dedo 0.86 (824, 251) (1019, 579)
Time spent: 2.167sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.96 (90, 625) (1080, 901)
Dedo 0.94 (649, 756) (907, 1257)
Time spent: 2.160sec
(416, 416, 3)
Found 2 boxes for i

Found 2 boxes for img
Panel 0.88 (19, 671) (945, 902)
Dedo 0.90 (379, 865) (515, 1146)
Time spent: 2.160sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.69 (99, 383) (255, 517)
Panel 0.93 (306, 347) (1841, 621)
Dedo 0.79 (885, 435) (1052, 853)
Time spent: 2.155sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.94 (68, 672) (983, 922)
Dedo 0.94 (518, 892) (768, 1271)
Time spent: 2.272sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.96 (32, 772) (1033, 1034)
Dedo 0.90 (434, 908) (623, 1266)
Time spent: 2.152sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.88 (21, 0) (1917, 553)
Dedo 0.85 (902, 274) (1137, 701)
Time spent: 2.174sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.94 (191, 0) (1746, 489)
Dedo 0.93 (639, 315) (783, 657)
Time spent: 2.181sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.58 (245, 41) (1714, 647)
Panel 0.65 (0, 208) (1920, 460)
Dedo 0.87 (973, 385) (1186, 654)
Time spent: 2.196sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.85 (341, 620) (1080, 894)
Time spent: 2.162sec
(4

Found 1 boxes for img
Dedo 0.94 (845, 334) (1042, 704)
Time spent: 2.167sec


/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide
100%|██████████| 1/1 [15:33<00:00, 933.82s/it]


## Generar CSV prueba completa (Grano grueso)

In [16]:
os.chdir('/home/ubuntu/tfm/utils/results')

rows = []
m = {'.pckl':'all', '_day.pckl': 'day', '_night.pckl': 'night'}
for s in ['.pckl', '_day.pckl', '_night.pckl']:
    for day in range(13, 22):
        data = pckl.load(open(str(day)+s, 'rb'))
        row ={
            'version': data[0][0],
            'mlflow': data[0][1],
            'result': data[2][0],
            'mAP': data[2][1].replace('mAP: ', '')
        }
        row['batch_size'] = row['mlflow'].data.params['batch_size']
        row['augmentation'] = row['mlflow'].data.params['augmentation']
        row['learning_date'] = float(row['mlflow'].data.params['learning_rate'])
        row['kind'] = m[s]
        del row['mlflow']
        rows.append(row)
final = []
for row in rows:
    first = f"{row['version']},{row['augmentation']},{row['batch_size']},{row['learning_date']},{row['kind']},{row['mAP']}"
    partial = []
    for _,r in row['result'].transpose().items():
        partial.append(','.join(list(map(lambda x:str(x), r))))
        #print(','.join(r[0]))
    second = ','.join(partial)
    final.append(','.join([first, second]))
for f in final:
    print(f)

13,False,4,0.0001,all,0.3812,Dedo,0.3829324831117263,0.43820224719101125,0.8082901554404145,356.0,156.0,37.0,200.0,Panel,0.3793993233311576,0.41551724137931034,0.8397212543554007,580.0,241.0,46.0,339.0
14,True,4,0.0001,all,0.7297,Dedo,0.8303311240100032,0.8932584269662921,0.895774647887324,356.0,318.0,37.0,38.0,Panel,0.6290790745162128,0.6741379310344827,0.8650442477876106,580.0,391.0,61.0,189.0
15,True,4,1e-06,all,0.0000,Dedo,0.0,-1,-1,356.0,-1,-1,356.0,Panel,0.0,-1,-1,580.0,-1,-1,580.0
16,True,4,1e-05,all,0.4879,Dedo,0.4488293183655009,0.5674157303370787,0.6917808219178082,356.0,202.0,90.0,154.0,Panel,0.5269231012964508,0.5775862068965517,0.7023060796645703,580.0,335.0,142.0,245.0
17,True,4,0.001,all,0.0135,Dedo,0.0,-1.0,-1.0,356.0,-1.0,-1.0,356.0,Panel,0.027079107505070994,0.027586206896551724,0.8888888888888888,580.0,16.0,2.0,564.0
18,True,8,0.0001,all,0.7246,Dedo,0.7902599471821179,0.851123595505618,0.8583569405099151,356.0,303.0,50.0,53.0,Panel,0.6589819074122285,0.68448275862068

## Generar CSV prueba completa (Grano fino)

In [15]:
os.chdir('/home/ubuntu/tfm/utils/results_2')

rows = []
for iou in ['0.6','0.7','0.8','0.9']:
    for day in ['14', '19', '21']:
        data = pckl.load(open(f"{day}_{iou}_.pckl", 'rb'))
        row ={
            'version': data[0][0],
            'mlflow': data[0][1],
            'result': data[2][0],
            'mAP': data[2][1].replace('mAP: ', '')
        }
        row['batch_size'] = row['mlflow'].data.params['batch_size']
        row['augmentation'] = row['mlflow'].data.params['augmentation']
        row['learning_date'] = float(row['mlflow'].data.params['learning_rate'])
        row['kind'] = iou
        del row['mlflow']
        rows.append(row)
final = []
for row in rows:
    first = f"{row['version']},{row['augmentation']},{row['batch_size']},{row['learning_date']},{row['kind']},{row['mAP']}"
    partial = []
    for _,r in row['result'].transpose().items():
        partial.append(','.join(list(map(lambda x:str(x), r))))
        #print(','.join(r[0]))
    second = ','.join(partial)
    final.append(','.join([first, second]))
for f in final:
    print(f)

14,True,4,0.0001,0.6,0.4855,Dedo,0.45096235137250285,0.6544943820224719,0.6563380281690141,356.0,233.0,122.0,123.0,Panel,0.5199668562601483,0.6051724137931035,0.7765486725663717,580.0,351.0,101.0,229.0
19,True,8,0.0001,0.6,0.5322,Dedo,0.4851024521853131,0.6657303370786517,0.6638655462184874,356.0,237.0,120.0,119.0,Panel,0.579343372616983,0.6379310344827587,0.8008658008658008,580.0,370.0,92.0,210.0
21,True,16,0.0001,0.6,0.5567,Dedo,0.5053196279647556,0.6882022471910112,0.6862745098039216,356.0,245.0,112.0,111.0,Panel,0.6080660745142735,0.65,0.8055555555555556,580.0,377.0,91.0,203.0
14,True,4,0.0001,0.7,0.2222,Dedo,0.091532775947198,0.2893258426966292,0.29014084507042254,356.0,103.0,252.0,253.0,Panel,0.3529251779264716,0.4810344827586207,0.6172566371681416,580.0,279.0,173.0,301.0
19,True,8,0.0001,0.7,0.2542,Dedo,0.11800129830465975,0.3258426966292135,0.32492997198879553,356.0,116.0,241.0,240.0,Panel,0.39049539394147187,0.496551724137931,0.6233766233766234,580.0,288.0,174.0,292.0
21,True,

## Versión 21 en detalle

In [33]:
os.chdir('/home/ubuntu/tfm/utils/results')
data_0_5 = pckl.load(open(f"21.pckl", 'rb'))
os.chdir('/home/ubuntu/tfm/utils/results_2')
data_0_6 = pckl.load(open(f"21_0.6_.pckl", 'rb'))
os.chdir('/home/ubuntu/tfm/utils/results_2')
data_0_7 = pckl.load(open(f"21_0.7_.pckl", 'rb'))
os.chdir('/home/ubuntu/tfm/utils/results_2')
data_0_8 = pckl.load(open(f"21_0.8_.pckl", 'rb'))
os.chdir('/home/ubuntu/tfm/utils/results_2')
data_0_9 = pckl.load(open(f"21_0.9_.pckl", 'rb'))

In [42]:
data_0_5[2][0][['label', 'AP', 'recall', 'precision', 'support', 'TP', 'FP']]

,label,AP,recall,precision,support,TP,FP
0,Dedo,0.900609,0.935393,0.932773,356.0,333.0,24.0
1,Panel,0.683016,0.701724,0.869658,580.0,407.0,61.0


In [41]:
data_0_5[2][0].columns

Index(['label', 'AP', 'recall', 'precision', 'support', 'TP', 'FP', 'FN'], dtype='object')

In [28]:
data_0_5[2][1]

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'mAP: 0.7918'

In [43]:
data_0_6[2][0][['label', 'AP', 'recall', 'precision', 'support', 'TP', 'FP']]

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,label,AP,recall,precision,support,TP,FP
0,Dedo,0.505320,0.688202,0.686275,356.0,245.0,112.0
1,Panel,0.608066,0.650000,0.805556,580.0,377.0,91.0


In [29]:
data_0_6[2][1]

'mAP: 0.5567'

In [44]:
data_0_7[2][0][['label', 'AP', 'recall', 'precision', 'support', 'TP', 'FP']]

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,label,AP,recall,precision,support,TP,FP
0,Dedo,0.123049,0.331461,0.330532,356.0,118.0,239.0
1,Panel,0.465045,0.555172,0.688034,580.0,322.0,146.0


In [30]:
data_0_7[2][1]

'mAP: 0.2940'

In [46]:
data_0_8[2][0][['label', 'AP', 'recall', 'precision', 'support', 'TP', 'FP']]

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,label,AP,recall,precision,support,TP,FP
0,Dedo,0.006899,0.070225,0.070028,356.0,25.0,332.0
1,Panel,0.162001,0.287931,0.356838,580.0,167.0,301.0


In [32]:
data_0_8[2][1]

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'mAP: 0.0844'

In [45]:
data_0_9[2][0][['label', 'AP', 'recall', 'precision', 'support', 'TP', 'FP']]

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,label,AP,recall,precision,support,TP,FP
0,Dedo,0.000048,0.005618,0.005602,356.0,2.0,355.0
1,Panel,0.011076,0.060345,0.074786,580.0,35.0,433.0


In [35]:
data_0_9[2][1]

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'mAP: 0.0056'